Part 2 - Applying ML to Cancer Data Analysis

For this project, you will create a machine learning model to predict the stage of cancer (I, II, III, or IV) from both RNA and protein-level gene expression for clear cell renal cell carcinoma (CCRCC) in CPTAC. Stage of cancer can be found using the tumor_stage_pathological column within the CPTAC clinical data. You can access the data the exact same way as BRCA, substituting the accession code.

1) Select what features to include in the model by finding the top 5 most differentially expressed proteins between Stage I and Stage III patients in CPTAC protein data. Repeat this process to find the top 5 most differential expression RNA between Stage I and Stage III patients in the CPTAC RNA data.
a) Use tumor_stage_pathological in the CPTAC clinical data.

2) Create a new dataframe of your selected features, where the rows are the patients and the columns are the expression values of genes you selected in step 1 (X data).

3) Create a separate list of the patients’ cancer stages, ie. tumor_stage_pathological (y data).

4) Scale and encode your features and target.
a) You need to do this! See the Hints for additional help.

5) Create a train test split of your X and y data with train_size=0.7.

6) Write code to test the accuracy of all 4 classification models we covered in this class (ie. KNeighborsClassifier, DecisionTreeClassifier, and MLPClassifier, GaussianNB). Since the accuracy of the models will change depending on the train-test split, you will need to run each model 10 times and find the average accuracy between all runs.

7) Compare the 4 mean accuracies and identify which model is best.

Hints:
● To find the most differentially expressed RNA and proteins, we will employ the same method as that in the Intro_to_CPTAC assignment. Find the column means for each gene and take the difference between the two groups.
● Protein data is already log scaled but RNA data is not. You will need to log scale the RNA data before comparing the differential expression. You can use np.log2() for this.
● For step 6: An example in Intro to ML includes the code to compare model performance, but only for regression models. However, the code can be used to compare classification models with a few alterations...

1) Select what features to include in the model by finding the top 5 most differentially expressed proteins between Stage I and Stage III patients in CPTAC protein data. Repeat this process to find the top 5 most differential expression RNA between Stage I and Stage III patients in the CPTAC RNA data.
a) Use tumor_stage_pathological in the CPTAC clinical data.

In [1]:
# Importing packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

# Setting up working directory
os.chdir('/Users/tomasmanea/desktop/QBIO490/qbio_490_tomas/analysis_data')

# 1. Import cptac
import cptac

# 2. Download the latest version of the cptac package
#cptac.download_latest_version()

# 3. Load the renal cancer data set
ccrcc = cptac.Ccrcc()

# Retrieving clinical, rna, protein data
clinical_data = ccrcc.get_clinical()
rna_data = ccrcc.get_transcriptomics()
protein_data = ccrcc.get_proteomics()

# Mask Creation
na_mask = clinical_data.loc[:, 'tumor_stage_pathological'].isna()
clinical_mask = clinical_data.loc[na_mask, :]
rna_mask = rna_data.loc[na_mask, :]
protein_mask = protein_data.loc[na_mask, :]

# Change 0s to NAs prior to removing **first rna, then protein
rna_masked = rna_mask.replace(0, np.nan)
gene_cleaned_masked = rna_masked.isna().sum() == 0
rna_masked = rna_masked.loc[:, gene_cleaned_masked]

protein_cleaned_masked = protein_mask.isna().sum() == 0
protein_masked = protein_mask.loc[:, protein_cleaned_masked]

# 4. Mask for Stage 1 and Stage 3
stage1_mask = clinical_mask.loc[:, 'tumor_stage_pathological'] == 'Stage I'
stage3_mask = clinical_mask.loc[:, 'tumor_stage_pathological'] == 'Stage III'

# Rna data requires logscale
rna_masked = np.log(rna_masked)

# 5. Find top 5 differential genes
# For RNA
stage1_rna = rna_masked.loc[stage1_mask, :]
stage3_rna = rna_masked.loc[stage3_mask, :]

# Calculate the mean difference between Stage I and Stage III for each gene
rna_mean_diff = np.abs(stage1_rna.mean() - stage3_rna.mean())

# Sort genes based on mean difference and select the top 5
stage1_t5_rna = rna_mean_diff.sort_values(ascending=False)[:5].index

# For Protein
stage1_proteins = protein_masked.loc[stage1_mask, :]
stage3_proteins = protein_masked.loc[stage3_mask, :]

# Calculate the mean difference between Stage I and Stage III for each protein
protein_mean_diff = np.abs(stage1_proteins.mean() - stage3_proteins.mean())

# Sort proteins based on mean difference and select the top 5
stage1_t5_proteins = protein_mean_diff.sort_values(ascending=False)[:5].index

# Display the selected genes and proteins
print("Top 5 RNA Genes:")
print(stage1_t5_rna)

print("\nTop 5 Protein Genes:")
print(stage1_t5_proteins)


cptac warning: Your version of cptac (0.9.1) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/var/folders/hp/p8lrjqpd7v35j_mgh8f2h_gc0000gn/T/ipykernel_2950/4119796738.py, line 13)


Top 5 RNA Genes:                          
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT'], dtype='object', name='Name')

Top 5 Protein Genes:
MultiIndex([('A1BG', 'NP_570602.2'),
            ('A1CF', 'NP_620310.1'),
            ( 'A2M', 'NP_000005.2'),
            ('AAAS', 'NP_056480.1'),
            ('AACS', 'NP_076417.2')],
           names=['Name', 'Database_ID'])


2) Create a new dataframe of your selected features, where the rows are the patients and the columns are the expression values of genes you selected in step 1 (X data).

In [3]:
# Selected Top 5 Genes
rna_selected_genes = ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT']
protein_selected_genes = ['A1BG', 'A1CF', 'A2M', 'AAAS', 'AACS']

# Combine the top 5 differentially expressed genes from RNA and protein data
selected_genes = rna_selected_genes + protein_selected_genes

# Create new dataframes for RNA and protein data with the selected features
rna_selected_df = rna_masked[selected_genes]
protein_selected_df = protein_masked[selected_genes]

# Display the new dataframes
print("RNA Selected Features:")
print(rna_selected_df.head())

print("\nProtein Selected Features:")
print(protein_selected_df.head())

# Combine RNA and protein selected features into X data
X_data = pd.concat([rna_selected_df, protein_selected_df], axis=1)


KeyError: "['A2ML1'] not in index"

3) Create a separate list of the patients’ cancer stages, ie. tumor_stage_pathological (y data).


In [4]:
# Selected Top 5 Genes
rna_selected_genes = ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT']
protein_selected_genes = ['A1BG', 'A1CF', 'A2M', 'AAAS', 'AACS']

# Combine the top 5 differentially expressed genes from RNA and protein data
selected_genes = list(stage1_t5_rna) + list(stage3_t5_rna) + list(stage1_t5_proteins) + list(stage3_t5_proteins)

# Create new dataframes for RNA and protein data with the selected features
rna_selected_df = rna_masked[selected_genes]
protein_selected_df = protein_masked[selected_genes]

# Display the new dataframes
print("RNA Selected Features:")
print(rna_selected_df.head())

print("\nProtein Selected Features:")
print(protein_selected_df.head())

# Create a separate list of the patients’ cancer stages (y data)
y_data = list(clinical_mask.loc[:, 'tumor_stage_pathological'])

# Display the first few elements of the y data
print("\nY Data (Tumor Stages):")
print(y_data[:5])


NameError: name 'stage3_t5_rna' is not defined

4) Scale and encode your features and target.
a) You need to do this! See the Hints for additional help.


In [5]:
#Packages
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Scale and encode features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_data)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_data)

NameError: name 'X_data' is not defined

5) Create a train test split of your X and y data with train_size=0.7.


In [6]:
#Train
#Import package
from sklearn.model_selection import train_test_split

#Train Split 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, train_size=0.7)
print(X_train.shape) # dataset split by train size
print(X_test.shape) # dataset not included in above dataset
print(y_train.shape) # labels for corresponding X_train
print(y_test.shape) # labels for corresponding X_test

NameError: name 'X_scaled' is not defined

6) Write code to test the accuracy of all 4 classification models we covered in this class (ie. KNeighborsClassifier, DecisionTreeClassifier, and MLPClassifier, GaussianNB). Since the accuracy of the models will change depending on the train-test split, you will need to run each model 10 times and find the average accuracy between all runs.


In [7]:
# Packages
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Function to test the accuracy of a classifier
def test_classifier(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Number of runs for averaging
num_runs = 10
accuracies_dt = []
accuracies_mlp = []
accuracies_knn = []
accuracies_nb = []

for _ in range(num_runs):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, train_size=0.7, random_state=_)

    # Test 1 - Decision Tree Classifier
    accuracy_dt = test_classifier(DecisionTreeClassifier(), X_train, X_test, y_train, y_test)
    accuracies_dt.append(accuracy_dt)

    # Test 2 - Multi-Layered Perceptron Classifier
    accuracy_mlp = test_classifier(MLPClassifier(), X_train, X_test, y_train, y_test)
    accuracies_mlp.append(accuracy_mlp)

    # Test 3 - KNeighbors Classifier
    accuracy_knn = test_classifier(KNeighborsClassifier(), X_train, X_test, y_train, y_test)
    accuracies_knn.append(accuracy_knn)

    # Test 4 - Gaussian Naive Bayes
    accuracy_nb = test_classifier(GaussianNB(), X_train, X_test, y_train, y_test)
    accuracies_nb.append(accuracy_nb)

# Display the average accuracies
print(f'Mean Accuracy - Decision Tree Classifier: {np.mean(accuracies_dt) * 100}%')
print(f'Mean Accuracy - Multi-Layered Perceptron Classifier: {np.mean(accuracies_mlp) * 100}%')
print(f'Mean Accuracy - KNeighbors Classifier: {np.mean(accuracies_knn) * 100}%')
print(f'Mean Accuracy - Gaussian Naive Bayes: {np.mean(accuracies_nb) * 100}%')


NameError: name 'X_scaled' is not defined

7) Compare the 4 mean accuracies and identify which model is best.

Comparison can be accomplished through looking at all aforementioned outputs and then sorting them.